<a href="https://colab.research.google.com/github/Merha23/AI_Tigrinya_Translation/blob/main/Final_Project_Source_Code_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

      **Integration GitHub with Google Colab**

In [1]:
# Clone the GitHub Repository in Google Colab
# Authenticate Google Colab with GitHub
# Clone with Authentication
# Generate a GitHub Personal Access Token (PAT)
# How to Use Your GitHub Token in Google Colab Securely
# cloning your repository using the token:
# Use the stored token to clone your GitHub repository in Google Colab:
# Clone the Repository Securely in Colab
# Instead of using your username/password, use the token as follows:

import os
from getpass import getpass

token = getpass('Enter your GitHub Personal Access Token: ')
os.environ["GITHUB_TOKEN"] = token

repo_url = f"https://{token}@github.com/Merha23/AI_Tigrinya_Translation.git"
!git clone {repo_url}
%cd AI_Tigrinya_Translation


Enter your GitHub Personal Access Token: ··········
Cloning into 'AI_Tigrinya_Translation'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 89 (delta 51), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 96.69 KiB | 3.87 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/AI_Tigrinya_Translation


In [2]:
# Run this python code and if cloning is successful, it should display the repository contents.

!ls


 Cheat_Sheet_Python_Code.ipynb		   LICENSE		      Practices.ipynb
 Final_Project_Source_Code_Program.ipynb  'Medical Translation.csv'   README.md


In [ ]:
# Mount Google Drive in Colab
# Run this command in Colab to access your Drive:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Pull the Latest Changes from GitHub
# Before pushing your changes, you need to sync your local copy of the repository with the remote one.
# Pull the latest changes:
# Run the following command to fetch and merge the latest changes from GitHub into your local branch:

!git pull https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

From https://github.com/Merha23/AI_Tigrinya_Translation
 * branch            main       -> FETCH_HEAD
Already up to date.


In [4]:
# Push Your Changes to GitHub
# After successfully pulling the latest changes and resolving any conflicts (if needed), you can now push your local changes:

!git push https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

Everything up-to-date


In [ ]:
# To reset to the remote state (undo local changes):

!git reset --hard origin/main


HEAD is now at 6c444c4 Created using Colab


In [ ]:
# To fetch the latest changes without merging them:

!git fetch


fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# Push Updates from Colab to GitHub
# After editing files, commit and push:

!git config --global user.email "halefomhailemariam19@gmail.com"
!git config --global user.name "Halefom Hailemariam"
!git add .
!git commit -m "Updated final project source code program"
!git push https://{os.environ['GITHUB_TOKEN']}@github.com/Merha23/AI_Tigrinya_Translation.git main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


    **Load the CSV File into a Pandas DataFrame**

In [5]:
import pandas as pd

df = pd.read_csv("Medical Translation.csv", encoding='utf-8')
df.head()  # Display first few rows

,id,english,tigrinya,domain
0,1,Information for parents,ንወለዲ ዝኸውን ሓበሬታ,medical
1,2,"If you choose not to vaccinate your child, und...",ውላድኩም ከይተኽትብዎ እንተመሪጽኩም፣ ሓደጋታትን ሓላፍነትን ተረድኡ,medical
2,3,Telling healthcare professionals your child’s ...,ንሰብ ሞያ ክንክን ጥዕና ኩነታት ክታበት ውላድኩም ምሕባር ብኽልተ ምኽንያ...,medical
3,4,If you choose to delay some vaccines or refuse...,ንገሊኡ ክታበታት ክተደናጉዩ ወይ ንገሊኡ ክታበታት ምሉእ ብምሉእ ክትነጽጉ...,medical
4,5,Please follow these steps to protect your chil...,ንውላድኩም፣ ንስድራቤትኩምን ንኻልኦትን ንምክልኻል በጃኹም ነዞም ስጉምትታ...,medical


     **Data Cleaning & Handling Missing Values**

     Before tokenization, we need to remove any unnecessary data such as missing values, duplicates, or improperly formatted sentences.

In [6]:
# Remove empty or missing values
df = df.dropna()

# Remove duplicates
df = df.drop_duplicates()

# Strip unwanted whitespace
df['english'] = df['english'].str.strip()
df['tigrinya'] = df['tigrinya'].str.strip()

print(f"Dataset size after cleaning: {df.shape}")

Dataset size after cleaning: (727, 4)


                **Sentence Tokenization**
  Since we are training a sequence-to-sequence model, we must tokenize each sentence.

In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Ensure column names match exactly
df['tokenized_english'] = df['english'].apply(lambda x: str(x).split('. '))
df['tokenized_tigrinya'] = df['tigrinya'].apply(lambda x: str(x).split('። '))
df[['tokenized_english', 'tokenized_tigrinya']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,tokenized_english,tokenized_tigrinya
0,[Information for parents],[ንወለዲ ዝኸውን ሓበሬታ]
1,"[If you choose not to vaccinate your child, un...",[ውላድኩም ከይተኽትብዎ እንተመሪጽኩም፣ ሓደጋታትን ሓላፍነትን ተረድኡ]
2,[Telling healthcare professionals your child’s...,[ንሰብ ሞያ ክንክን ጥዕና ኩነታት ክታበት ውላድኩም ምሕባር ብኽልተ ምኽን...
3,[If you choose to delay some vaccines or refus...,[ንገሊኡ ክታበታት ክተደናጉዩ ወይ ንገሊኡ ክታበታት ምሉእ ብምሉእ ክትነጽ...
4,[Please follow these steps to protect your chi...,[ንውላድኩም፣ ንስድራቤትኩምን ንኻልኦትን ንምክልኻል በጃኹም ነዞም ስጉምት...


                **Check Sentence Length Distribution**

We need to analyze the length of sentences to ensure they fit within the model's input constraints.

In [8]:
# Check sentence length distribution
df['eng_length'] = df['english'].apply(lambda x: len(str(x).split()))
df['tir_length'] = df['tigrinya'].apply(lambda x: len(str(x).split()))

df[['eng_length', 'tir_length']].describe()

,eng_length,tir_length
count,727.000000,727.000000
mean,10.493810,9.477304
std,10.393082,8.752440
min,1.000000,1.000000
25%,2.000000,3.000000
50%,7.000000,7.000000
75%,16.000000,14.000000
max,69.000000,62.000000


            **Filtering Extremely Long or Short Sentences**

Very short or long sentences might reduce model performance. We filter out sentences that are too short (<3 words) or too long (>128 words).

In [9]:
# Filter out sentences that are too short or too long
df = df[(df['eng_length'] >= 3) & (df['eng_length'] <= 128)]
df = df[(df['tir_length'] >= 3) &  (df['tir_length'] <= 128)]

print(f"Dataset size after filtering: {df.shape}")

Dataset size after filtering: (538, 8)


In [15]:
print(df.columns)  # Verify column names

Index(['id', 'english', 'tigrinya', 'domain', 'tokenized_english',
       'tokenized_tigrinya', 'eng_length', 'tir_length'],
      dtype='object')


               **Save the Processed Dataset**

After preprocessing, we save the cleaned and tokenized dataset for use in model training.

In [10]:
df.to_csv("cleaned_Medical Translation.csv", index=False, encoding='utf-8')
print("Dataset saved successfully!")

Dataset saved successfully!


In [11]:
import os
print(os.getcwd())  # Shows current directory
print(os.listdir())  # Lists all files in the directory


/content/AI_Tigrinya_Translation
['Final_Project_Source_Code_Program.ipynb', '.git', 'cleaned_Medical Translation.csv', 'README.md', 'LICENSE', '.gitignore', 'Practices.ipynb', 'Medical Translation.csv', 'Cheat_Sheet_Python_Code.ipynb']


In [12]:
from google.colab import files
files.download("cleaned_Medical Translation.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

      **Fine-Tuning NLLB-200 Model **

Install and Load Hugging Face Transformers

In [13]:
!pip install transformers datasets torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Load Pretrained NLLB-200 Model

In [16]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Prepare Data for Fine-Tuning

In [17]:
from datasets import Dataset

def preprocess_function(examples):
    inputs = tokenizer(examples["english"], max_length=128, truncation=True)
    targets = tokenizer(examples["tigrinya"], max_length=128, truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs

dataset =Dataset.from_pandas(df)
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/538 [00:00<?, ? examples/s]

        ** Train the Model**

In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-18-72fd59edbee6>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: You have set `args.eval_strategy` to IntervalStrategy.EPOCH but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 